<a href="https://colab.research.google.com/github/rahulj004/Groq-Assignment-Repo/blob/main/Conversation_Management_%26_Classification_using_Groq_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq openai -q

In [ ]:
import os
from groq import Groq
import json


GROQ_API_KEY = "Enter your grok API here"


client = Groq(api_key=GROQ_API_KEY)


MODEL = 'llama3-8b-8192'

In [ ]:
class ConversationManager:

    def __init__(self, client, model, summarize_after_k_turns=5):
        self.client = client
        self.model = model
        self.history = []
        self.summarize_after_k_turns = summarize_after_k_turns
        self.turn_counter = 0

    def _summarize_conversation(self, conversation_text):

        print("\n[SYSTEM] Summarizing conversation...")
        system_prompt = "Summarize the following conversation concisely. Capture the key topics, user intent, and any important entities or decisions made. The summary should serve as a memory for the assistant."

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": conversation_text}
                ],
                temperature=0.3,
            )
            summary = response.choices[0].message.content
            print("[SYSTEM] Summary generated.")
            return summary
        except Exception as e:
            print(f"[SYSTEM] Error during summarization: {e}")
            return "Error: Could not generate summary."

    def add_message(self, role, content):

        self.history.append({"role": role, "content": content})
        if role == 'assistant':
            self.turn_counter += 1


        if self.turn_counter > 0 and self.turn_counter % self.summarize_after_k_turns == 0:

            last_message = self.history[-1]
            history_to_summarize = self.history[:-1]


            conversation_str = "\n".join([f"{msg['role']}: {msg['content']}" for msg in history_to_summarize])


            summary = self._summarize_conversation(conversation_str)


            self.history = [
                {"role": "system", "content": f"This is a summary of the previous conversation: {summary}"},
                last_message
            ]
            print("\n[SYSTEM] Conversation history has been summarized and replaced.")

    def get_history(self):
        return self.history

    def get_truncated_history(self, max_turns=None, max_chars=None):
        if max_turns:

            num_messages = max_turns * 2
            return self.history[-num_messages:]

        if max_chars:
            truncated = []
            current_chars = 0

            for message in reversed(self.history):
                msg_len = len(message['content'])
                if current_chars + msg_len > max_chars:
                    break
                truncated.insert(0, message)
                current_chars += msg_len
            return truncated

        return self.history

In [ ]:

conversation = ConversationManager(client, MODEL, summarize_after_k_turns=3)

sample_chats = [
    ("user", "Hi, I'd like to book a flight to Tokyo."),
    ("assistant", "Of course! When would you like to travel?"),
    ("user", "I'm thinking of leaving around October 20th and coming back on the 30th."),
    ("assistant", "Great. Are you flying from Pune?"),
    ("user", "Yes, that's right. Pune to Tokyo."),
    ("assistant", "Got it. Let me check for flights from Pune (PNQ) to Tokyo (NRT) departing on Oct 20th and returning Oct 30th. One moment."),
    ("assistant", "Certainly. Searching for business class availability. I found a flight with Japan Airlines."),
    ("user", "What's the price?"),
    ("assistant", "The round trip in business class is approximately ₹1,80,000."),
    ("user", "Okay, that's a bit high. What about premium economy?"),
    ("assistant", "Checking for premium economy... Yes, there's an option for ₹1,10,000 with Singapore Airlines."),

for i, (role, content) in enumerate(sample_chats):
    print(f"\n--- Adding Message {i+1} ---")
    print(f"{role.title()}: {content}")
    conversation.add_message(role, content)


    print("\n--- Current Conversation History ---")
    print(json.dumps(conversation.get_history(), indent=2))
    print("-" * 30)


--- Adding Message 1 ---
User: Hi, I'd like to book a flight to Tokyo.

--- Current Conversation History ---
[
  {
    "role": "user",
    "content": "Hi, I'd like to book a flight to Tokyo."
  }
]
------------------------------

--- Adding Message 2 ---
Assistant: Of course! When would you like to travel?

--- Current Conversation History ---
[
  {
    "role": "user",
    "content": "Hi, I'd like to book a flight to Tokyo."
  },
  {
    "role": "assistant",
    "content": "Of course! When would you like to travel?"
  }
]
------------------------------

--- Adding Message 3 ---
User: I'm thinking of leaving around October 20th and coming back on the 30th.

--- Current Conversation History ---
[
  {
    "role": "user",
    "content": "Hi, I'd like to book a flight to Tokyo."
  },
  {
    "role": "assistant",
    "content": "Of course! When would you like to travel?"
  },
  {
    "role": "user",
    "content": "I'm thinking of leaving around October 20th and coming back on the 30th."
  

In [ ]:
user_details_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "The full name of the user."},
        "email": {"type": "string", "description": "The email address of the user."},
        "phone": {"type": "string", "description": "The phone number of the user."},
        "location": {"type": "string", "description": "The city or address of the user."},
        "age": {"type": "integer", "description": "The age of the user."},
    },
    "required": ["name", "email"],
}

tools = [
    {
        "type": "function",
        "function": {
            "name": "extract_user_info",
            "description": "Extracts user information from the chat.",
            "parameters": user_details_schema,
        },
    }
]


def extract_user_details(chat_content):

    print(f"\n--- Parsing Chat ---")
    print(f"Input: '{chat_content}'")

    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": chat_content}],
            tools=tools,
            tool_choice="auto",
            temperature=0.0,
        )
        message = response.choices[0].message
        tool_calls = message.tool_calls

        if tool_calls:
            function_args_str = tool_calls[0].function.arguments
            extracted_data = json.loads(function_args_str)
            return extracted_data
        else:
            print("Model did not find any user details to extract.")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


sample_chats_for_extraction = [

    "Hi, my name is Arun Kumar. I'm 28 years old and live in Pune. You can reach me at arun.k@example.com or on my mobile at +91-9876543210.",

    "Please sign me up. My name is Priya Sharma and my email is priya.sharma@email.com. My phone is 8887776665.",

    "Okay, let's proceed. I'm Vikram Singh from Mumbai. I'm 35. My email is vikram.s@workplace.net if you need it.",

    "What is the weather like today?"
]


for chat in sample_chats_for_extraction:
    extracted_info = extract_user_details(chat)

    if extracted_info:
        print("\n--- Extracted JSON Output ---")
        print(json.dumps(extracted_info, indent=2))
    print("-" * 30)


--- Parsing Chat ---
Input: 'Hi, my name is Arun Kumar. I'm 28 years old and live in Pune. You can reach me at arun.k@example.com or on my mobile at +91-9876543210.'
An error occurred: Error code: 400 - {'error': {'message': 'The model `llama3-8b-8192` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}
------------------------------

--- Parsing Chat ---
Input: 'Please sign me up. My name is Priya Sharma and my email is priya.sharma@email.com. My phone is 8887776665.'
An error occurred: Error code: 400 - {'error': {'message': 'The model `llama3-8b-8192` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}
----------------------------